In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import missingno as msno

from time import sleep

from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from tqdm import tqdm


from sklearn.model_selection import cross_validate

from sklearn.model_selection import ParameterGrid

import warnings
warnings.filterwarnings('ignore')

import math

from sklearn import ensemble

pd.set_option('display.max_columns', None)

%matplotlib inline  

housing_raw = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
price = housing_raw['SalePrice'].copy()
housing_raw

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
5,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,903205040,952,121000,30,RL,NaN,8854,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Unf,6,6,1916,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,952.0,952.0,Grav,Fa,N,FuseF,952,0,0,0.0,0.0,1,0,2,1,Fa,4,Typ,1,Gd,Detchd,1916.0,Unf,1.0,192.0,Fa,Po,P,0,98,0,0,40,0,NaN,NaN,NaN,0,5,2009,WD,Normal
760,905402060,1733,139600,20,RL,NaN,13680,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Edwards,Norm,Norm,1Fam,1Story,3,5,1955,1955,Hip,CompShg,BrkFace,Wd Sdng,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,Ex,Y,FuseA,1733,0,0,0.0,0.0,2,0,4,1,TA,8,Min2,1,Gd,Attchd,1955.0,Unf,2.0,452.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
761,909275030,2002,145000,90,RH,82.0,6270,Pave,NaN,Reg,HLS,AllPub,Inside,Gtl,Crawfor,Norm,Norm,Duplex,2Story,5,6,1949,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,284.0,Unf,0.0,717.0,1001.0,GasA,TA,N,FuseA,1001,1001,0,0.0,0.0,2,0,4,2,TA,8,Typ,0,NaN,2Types,1949.0,Unf,3.0,871.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
762,907192040,1842,217500,60,RL,N

# Eliminating Outliers

In [2]:
housing_raw = housing_raw[housing_raw['GrLivArea']< 4000]

In [3]:
housing_raw['MSSubClass'] = housing_raw['MSSubClass'].astype('str')
housing_raw['YrSold'] = housing_raw['YrSold'].astype(str)
housing_raw['MoSold'] = housing_raw['MoSold'].astype(str)

# Filling in NA for Numerical Columns

In [4]:
df_num = housing_raw.select_dtypes(['int64','float64']).copy()
df_num.head()

,PID,GrLivArea,SalePrice,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal
1,909176150,856,126000,NaN,7890,6,6,1939,1950,0.0,238.0,0.0,618.0,856.0,856,0,0,1.0,0.0,1,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,0
2,905476230,1049,139500,42.0,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0
3,911128020,1001,124900,60.0,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0
4,535377150,1039,114000,80.0,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,405.0,717,322,0,0.0,0.0,1,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,0
5,534177230,1665,227000,70.0,8400,8,6,2001,2001,0.0,643.0,0.0,167.0,810.0,810,855,0,1.0,0.0,2,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,0


In [5]:
df_num['LotFrontage'].fillna(df_num['LotFrontage'].mean(), inplace = True)

# Because there is only 1 null value for area, we just fill in with the Year the house was built
df_num.loc[df_num['GarageArea'].isnull(), 'GarageYrBlt'] = df_num.loc[df_num['GarageArea'].isnull(),'YearBuilt']
df_num['GarageArea'].fillna(housing_raw[housing_raw['GarageType']== 'Detchd']['GarageArea'].mean(), inplace = True)
df_num.loc[df_num['GarageCars'].isnull(), 'GarageCars'] = 2


df_num.drop(['SalePrice', 'PID'],inplace=True, axis = 1)
# MsnrVrArea, TotalBsmtSF, GarageCars, BsmtHalfBath, BsmtFullBath
df_num.fillna(0,inplace = True)
price = housing_raw['SalePrice'].copy()
# msno.bar(df_num)
df_num

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal
1,856,68.470227,7890,6,6,1939,1950,0.0,238.0,0.0,618.0,856.0,856,0,0,1.0,0.0,1,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,0
2,1049,42.000000,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0
3,1001,60.000000,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0
4,1039,80.000000,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,405.0,717,322,0,0.0,0.0,1,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,0
5,1665,70.000000,8400,8,6,2001,2001,0.0,643.0,0.0,167.0,810.0,810,855,0,1.0,0.0,2,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,952,68.470227,8854,6,6,1916,1950,0.0,0.0,0.0,952.0,952.0,952,0,0,0.0,0.0,1,0,2,1,4,1,1916.0,1.0,192.0,0,98,0,0,40,0,0
760,1733,68.470227,13680,3,5,1955,1955,0.0,0.0,0.0,0.0,0.0,1733,0,0,0.0,0.0,2,0,4,1,8,1,1955.0,2.0,452.0,0,0,0,0,0,0,0
761,2002,82.000000,6270,5,6,1949,1950,0.0,284.0,0.0,717.0,1001.0,1001,1001,0,0.0,0.0,2,0,4,2,8,0,1949.0,3.0,871.0,0,0,0,0,0,0,0
762,1842,68.470227,8826,7,5,2000,2000,144.0,841.0,0.0,144.0,985.0,985,857,0,1.0,0.0,2,1,3,1,7,1,2000.0,2.0,486.0,193,96,0,0,0,0,0


# One Hot Encoding Categorical Columns and filling in NA

In [6]:
# Selecting Categorical Values
# Mapping the respective categorical columns ordinally
df_cat = housing_raw.select_dtypes(['object']).copy()
df_cat.fillna('None', inplace = True)
df_cat
# Define the mapping
po_ex_ordinal_mapping = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['ExterQual'] = df_cat['ExterQual'].map(po_ex_ordinal_mapping)
df_cat['ExterCond'] = df_cat['ExterCond'].map(po_ex_ordinal_mapping)
df_cat['KitchenQual'] = df_cat['KitchenQual'].map(po_ex_ordinal_mapping)
df_cat['HeatingQC'] = df_cat['HeatingQC'].map(po_ex_ordinal_mapping)


# Defind the mapping
none_ex6_ordinal_mapping = {'None': 0, 'Po': 1, 'Fa': 2, 'TA':3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['BsmtQual'] = df_cat['BsmtQual'].map(none_ex6_ordinal_mapping)
df_cat['BsmtCond'] = df_cat['BsmtCond'].map(none_ex6_ordinal_mapping)
df_cat['FireplaceQu'] = df_cat['FireplaceQu'].map(none_ex6_ordinal_mapping)
df_cat['GarageQual'] = df_cat['GarageQual'].map(none_ex6_ordinal_mapping)
df_cat['GarageCond'] = df_cat['GarageCond'].map(none_ex6_ordinal_mapping)


# Define the mapping
none_fin_ordinal_mapping = {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin':3}
# Applying the mapping
df_cat['GarageFinish'] = df_cat['GarageFinish'].map(none_fin_ordinal_mapping)


# Define the mapping
none_ex5_ordinal_mapping = {'None': 0, 'Fa': 1, 'TA': 2, 'Gd':3, 'Ex':4}
# Applying the mapping
df_cat['PoolQC'] = df_cat['PoolQC'].map(none_ex5_ordinal_mapping)

# Define mapping
binary_mapping = {'Y': 1, 'N':0}
df_cat['CentralAir'] = df_cat['CentralAir'].map(binary_mapping)

# 'BsmtCond',
#  'BsmtQual',
#  'CentralAir',
#  'ExterCond',
#  'ExterQual',
#  'FireplaceQu',
#  'GarageCond',
#  'GarageFinish',
#  'GarageQual',
#  'HeatingQC',
#  'KitchenQual',
#  'PoolQC'

df_cat.shape

(2578, 46)

In [7]:
df_cat

,MSSubClass,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
1,30,RL,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,Gable,CompShg,Wd Sdng,Wd Sdng,None,3,3,CBlock,3,3,No,Rec,Unf,GasA,3,1,SBrkr,3,Typ,4,Detchd,1,3,3,Y,0,None,None,3,2010,WD,Normal
2,120,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,Gable,CompShg,HdBoard,HdBoard,BrkFace,4,3,CBlock,4,3,Mn,GLQ,ALQ,GasA,3,1,SBrkr,4,Typ,0,Attchd,3,3,3,Y,0,None,None,2,2009,WD,Normal
3,30,C (all),Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,4,3,BrkTil,3,3,No,ALQ,Unf,GasA,5,1,SBrkr,4,Typ,0,Detchd,1,3,1,N,0,None,None,11,2007,WD,Normal
4,70,RL,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,Gable,CompShg,MetalSd,MetalSd,None,4,4,BrkTil,2,3,No,Unf,Unf,GasA,4,1,SBrkr,3,Typ,0,Detchd,1,3,3,N,0,None,None,5,2009,WD,Normal
5,60,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,4,3,PConc,4,3,No,GLQ,Unf,GasA,5,1,SBrkr,4,Typ,0,Attchd,3,3,3,Y,0,None,None,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,30,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Unf,Gable,CompShg,Wd Sdng,Wd Sdng,None,3,3,BrkTil,3,3,No,Unf,Unf,Grav,2,0,FuseF,2,Typ,4,Detchd,1,2,1,P,0,None,None,5,2009,WD,Normal
760,20,RL,Pave,None,IR1,Lvl,AllPub,CulDSac,Gtl,Edwards,Norm,Norm,1Fam,1Story,Hip,CompShg,BrkFace,Wd Sdng,None,3,3,Slab,0,0,None,None,None,GasA,5,1,FuseA,3,Min2,4,Attchd,1,3,3,Y,0,None,None,6,2009,WD,Normal
761,90,RH,Pave,None,Reg,HLS,AllPub,Inside,Gtl,Crawfor,Norm,Norm,Duplex,2Story,Gable,CompShg,MetalSd,MetalSd,None,3,3,CBlock,3,3,No,BLQ,Unf,GasA,3,0,FuseA,3,Typ,0,2Types,1,3,3,Y,0,None,None,8,2007,WD,Normal
762,60,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,4,3,PConc,4,3,No,GLQ,Unf,GasA,5,1,SBrkr,4,Typ,3,Attchd,3,3,3,Y,0,None,None,7,2007,WD,Normal


In [8]:
df_no_dummy = pd.concat([df_num, df_cat], axis = 1)
df_no_dummy

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
1,856,68.470227,7890,6,6,1939,1950,0.0,238.0,0.0,618.0,856.0,856,0,0,1.0,0.0,1,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,0,30,RL,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,Gable,CompShg,Wd Sdng,Wd Sdng,None,3,3,CBlock,3,3,No,Rec,Unf,GasA,3,1,SBrkr,3,Typ,4,Detchd,1,3,3,Y,0,None,None,3,2010,WD,Normal
2,1049,42.000000,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0,120,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,Gable,CompShg,HdBoard,HdBoard,BrkFace,4,3,CBlock,4,3,Mn,GLQ,ALQ,GasA,3,1,SBrkr,4,Typ,0,Attchd,3,3,3,Y,0,None,None,2,2009,WD,Normal
3,1001,60.000000,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0,30,C (all),Pave,None,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,None,4,3,BrkTil,3,3,No,ALQ,Unf,GasA,5,1,SBrkr,4,Typ,0,Detchd,1,3,1,N,0,None,None,11,2007,WD,Normal
4,1039,80.000000,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,405.0,717,322,0,0.0,0.0,1,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,0,70,RL,Pave,None,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,Gable,CompShg,MetalSd,MetalSd,None,4,4,BrkTil,2,3,No,Unf,Unf,GasA,4,1,SBrkr,3,Typ,0,Detchd,1,3,3,N,0,None,None,5,2009,WD,Normal
5,1665,70.000000,8400,8,6,2001,2001,0.0,643.0,0.0,167.0,810.0,810,855,0,1.0,0.0,2,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,0,60,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,4,3,PConc,4,3,No,GLQ,Unf,GasA,5,1,SBrkr,4,Typ,0,Attchd,3,3,3,Y,0,None,None,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,952,68.470227,8854,6,6,1916,1950,0.0,0.0,0.0,952.0,952.0,952,0,0,0.0,0.0,1,0,2,1,4,1,1916.0,1.0,192.0,0,98,0,0,40,0,0,30,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Unf,Gable,CompShg,Wd Sdng,Wd Sdng,None,3,3,BrkTil,3,3,No,Unf,Unf,Grav,2,0,FuseF,2,Typ,4,Detchd,1,2,1,P,0,None,None,5,2009,WD,Normal
760,1733,68.470227,13680,3,5,1955,1955,0.0,0.0,0.0,0.0,0.0,1733,0,0,0.0,0.0,2,0,4,1,8,1,1955.0,2.0,452.0,0,0,0,0,0,0,0,20,RL,Pave,None,IR1,Lvl,AllPub,CulDSac,Gtl,Edwards,Norm,Norm,1Fam,1Story,Hip,CompShg,BrkFace,Wd Sdng,None,3,3,Slab,0,0,None,None,None,GasA,5,1,FuseA,3,Min2,4,Attchd,1,3,3,Y,0,None,None,6,2009,WD,Normal
761,2002,82.000000,6270,5,6,1949,1950,0.0,284.0,0.0,717.0,1001.0,1001,1001,0,0.0,0.0,2,0,4,2,8,0,1949.0,3.0,871.0,0,0,0,0,0,0,0,90,RH,Pave,None,Reg,HLS,AllPub,Inside,Gtl,Crawfor,Norm,Norm,Duplex,2Story,Gable,CompShg,MetalSd,MetalSd,None,3,3,CBlock,3,3,No,BLQ,Unf,GasA,3,0,FuseA,3,Typ,0,2Types,1,3,3,Y,0,None,None,8,2007,WD,Normal
762,1842,68.470227,8826,7,5,2000,2000,144.0,841.0,0.0,144.0,985.0,985,857,0,1.0,0.0,2,1,3,1,7,1,2000.0,2.0,486.0,193,96,0,0,0,0,0,60,RL,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,

In [9]:
df_cat_dummy = df_no_dummy.select_dtypes('object')
df_num_dummy = df_no_dummy.select_dtypes(['int64','float64'])
df_cat_dummy = pd.get_dummies(df_cat_dummy)
df_cat_dummy

,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_A (agr),MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSewr,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_None,Electrical_SBrkr,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_Gar2,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_1,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_VWD,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloc

# Concatenating Num and Cat Cols for Final Df

In [10]:
df = pd.concat([df_num_dummy, df_cat_dummy], axis = 1)
df.columns

Index(['GrLivArea', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'SaleType_New', 'SaleType_Oth', 'SaleType_VWD', 'SaleType_WD ',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=293)

In [11]:
df

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_A (agr),MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSewr,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_None,Electrical_SBrkr,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_G

In [12]:
df_num_unscaled = df.select_dtypes(['int64','float64'])
df_cat_dummy = df.select_dtypes('uint8')
df_num_unscaled

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC
1,856,68.470227,7890,6,6,1939,1950,0.0,238.0,0.0,618.0,856.0,856,0,0,1.0,0.0,1,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,0,3,3,3,3,3,1,3,4,1,3,3,0
2,1049,42.000000,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0,4,3,4,3,3,1,4,0,3,3,3,0
3,1001,60.000000,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0,4,3,3,3,5,1,4,0,1,3,1,0
4,1039,80.000000,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,405.0,717,322,0,0.0,0.0,1,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,0,4,4,2,3,4,1,3,0,1,3,3,0
5,1665,70.000000,8400,8,6,2001,2001,0.0,643.0,0.0,167.0,810.0,810,855,0,1.0,0.0,2,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,0,4,3,4,3,5,1,4,0,3,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,952,68.470227,8854,6,6,1916,1950,0.0,0.0,0.0,952.0,952.0,952,0,0,0.0,0.0,1,0,2,1,4,1,1916.0,1.0,192.0,0,98,0,0,40,0,0,3,3,3,3,2,0,2,4,1,2,1,0
760,1733,68.470227,13680,3,5,1955,1955,0.0,0.0,0.0,0.0,0.0,1733,0,0,0.0,0.0,2,0,4,1,8,1,1955.0,2.0,452.0,0,0,0,0,0,0,0,3,3,0,0,5,1,3,4,1,3,3,0
761,2002,82.000000,6270,5,6,1949,1950,0.0,284.0,0.0,717.0,1001.0,1001,1001,0,0.0,0.0,2,0,4,2,8,0,1949.0,3.0,871.0,0,0,0,0,0,0,0,3,3,3,3,3,0,3,0,1,3,3,0
762,1842,68.470227,8826,7,5,2000,2000,144.0,841.0,0.0,144.0,985.0,985,857,0,1.0,0.0,2,1,3,1,7,1,2000.0,2.0,486.0,193,96,0,0,0,0,0,4,3,4,3,5,1,4,3,3,3,3,0


# Scaling Numerical Columns

In [13]:
scaler = StandardScaler()
scaler.fit(df_num_unscaled)
scaled_num_cols = scaler.transform(df_num_unscaled)
scaled_df_num = pd.DataFrame(scaled_num_cols, columns = df_num_unscaled.columns, index = df_num_unscaled.index)
scaled_df_num

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC
1,-1.303728,0.000000,-0.273213,-0.031580,0.339802,-1.053092,-1.646773,-0.564369,-0.479750,-0.305351,0.186773,-0.430024,-0.771104,-0.794860,-0.095647,1.089629,-0.253009,-1.009580,-0.757088,-1.033035,-0.199131,-1.557024,0.611682,0.140002,0.343282,-0.325878,-0.742741,-0.699388,-0.362343,-0.099340,2.635518,-0.054813,-0.097769,-0.662480,-0.248625,-0.510089,0.133479,-1.188897,0.259798,-0.748254,1.254384,-0.798986,0.267572,0.257405,-0.054241
2,-0.902872,-1.282083,-0.724230,-0.765034,-0.551357,0.461396,0.012780,0.292857,0.254809,1.947821,-1.022106,0.033831,-0.253815,-0.794860,-0.095647,1.089629,-0.253009,0.827186,-0.757088,-1.033035,-0.199131,-0.903915,-0.931289,0.244626,-1.011971,-0.967797,-0.742741,0.898982,-0.362343,-0.099340,-0.285269,-0.054813,-0.097769,1.129622,-0.248625,0.623540,0.133479,-1.188897,0.259798,0.816222,-0.987031,1.455842,0.267572,0.257405,-0.054241
3,-1.002567,-0.410255,-0.499030,-0.765034,3.013279,-1.355989,1.135419,-0.564369,0.687590,-0.305351,-1.031514,-0.475688,-0.382467,-0.794860,-0.095647,-0.838349,-0.253009,-1.009580,-0.757088,-1.033035,-0.199131,-0.903915,-0.931289,0.119077,-1.011971,-1.209119,0.451659,-0.699388,0.292709,3.300038,-0.285269,-0.054813,-0.097769,1.129622,-0.248625,-0.510089,0.133479,0.903237,0.259798,0.816222,-0.987031,-0.798986,0.267572,-2.653078,-0.054241
4,-0.923642,0.558444,-0.241624,-1.498489,2.122120,-2.365648,0.940177,-0.564369,-1.036517,-0.305351,-0.314183,-1.513953,-1.143659,-0.032572,-0.095647,-0.838349,-0.253009,-1.009580,-0.757088,-1.033035,-0.199131,-0.250806,-0.931289,0.142327,-1.011971,-0.895400,-0.742741,-0.699388,2.257867,-0.099340,1.667787,-0.054813,-0.097769,1.129622,2.410938,-1.643719,0.133479,-0.142830,0.259798,-0.748254,-0.987031,-0.798986,0.267572,0.257405,-0.054241
5,0.376546,0.074094,-0.210281,1.435328,0.339802,1.033536,0.842556,-0.564369,0.467690,-0.305351,-0.873936,-0.540580,-0.894396,1.229228,-0.095647,1.089629,-0.253009,0.827186,1.246785,0.182467,-0.199131,-0.250806,-0.931289,0.284151,0.343282,0.296734,-0.742741,-0.014372,-0.362343,-0.099340,-0.285269,-0.054813,-0.097769,1.129622,-0.248625,0.623540,0.133479,0.903237,0.259798,0.816222,-0.987031,1.455842,0.267572,0.257405,-0.054241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,-1.104339,0.000000,-0.154259,-0.031580,0.339802,-1.827163,-1.646773,-0.564369,-1.036517,-0.305351,0.972309,-0.199298,-0.513800,-0.794860,-0.095647,-0.838349,-0.253009,-1.009580,-0.757088,-1.033035,-0.199131,-1.557024,0.611682,0.086527,-1.011971,-1.324954,-0.742741,0.792424,-0.362343,-0.099340,0.418535,-0.054813,-0.097769,-0.662480,-0.248625,-0.510089,0.133479,-2.234964,-3.849149,-2.312730,1.254384,-0.798986,-1.178551,-2.653078,-0.054241
760,0.517780,0.000000,0.441256,-2.231943,-0.551357,-0.514607,-1.402721,-0.564369,-1.036517,-0.305351,-1.266704,-2.487327,1.579478,-0.794860,-0.095647,-0.838349,-0.253009,0.827186,-0.757088,1.397968,-0.199131,1.055412,0.611682,0.177202,0.343282,-0.070076,-0.742741,-0.699388,-0.362343,-0.099340,-0.285269,-0.054813,-0.097769,-0.662480,-0.248625,-3.910978,-5.187781,0.903237,0.259798,-0.748254,1.254384,-0.798986,0.267572,0.257405,-0.054241
761,1.076487,0.655313,-0.473117,-0.765034,0.339802,-0.716539,-1.646773,-0.564369,-0.372140,-0.305351,0.419612,-0.081532,-0.382467,1.574862,-0.095647,-0.838349,-0.253009,0.827186,-0.757088,1.397968,4.737030,1.055412,-0.931289,0.163252,1.698534,1.952207,-0.7427

In [14]:
scaled_df = pd.concat([scaled_df_num,df_cat_dummy],axis=1)
scaled_df

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_A (agr),MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSewr,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_None,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_None,Electrical_SBrkr,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_G

# Defining X and Y

In [15]:
x = scaled_df
y = price

In [16]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

# Standard Lasso with $\alpha$ = 1.0

In [18]:
lasso = Lasso()
lasso.fit(X_train, y_train)
lasso.score(X_test, y_test)

0.9136383008526572

In [19]:
cross_val_score_list = cross_val_score(lasso, x, y, cv=kf)

print('Avg Score: ',np.mean(cross_val_score_list))
print(cross_val_score_list)
print('variance:',np.var(cross_val_score_list))

Avg Score:  0.911176214002537
[0.9136383  0.90948072 0.92193689 0.91133202 0.89949314]
variance: 5.224943623285341e-05


In [20]:
%%time
lasso = Lasso()
# Initialize lists to store results
scores_list = []
coefficients_list = []
lasso.set_params(alpha=1)
fold_num=1
# Perform Grid Search in each fold
for train_index, test_index in kf.split(x):
    
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    lasso.fit(X_train, y_train)
    score = lasso.score(X_test, y_test)
    print('test.score:',score)
    y_pred = lasso.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print('r2 manual:',r2)
    print(f'Fold {fold_num} score:', score)

    # Store best parameters and score
    scores_list.append({
        'score':r2,
    })
    # Store coefficients of the best model
    coefficients_list.append({
        'coef': lasso.coef_,
    })
    fold_num += 1

test.score: 0.9136383008526572
r2 manual: 0.9136383008526572
Fold 1 score: 0.9136383008526572
test.score: 0.9094807211858146
r2 manual: 0.9094807211858146
Fold 2 score: 0.9094807211858146
test.score: 0.9219368898607594
r2 manual: 0.9219368898607594
Fold 3 score: 0.9219368898607594
test.score: 0.9113320170837838
r2 manual: 0.9113320170837838
Fold 4 score: 0.9113320170837838
test.score: 0.8994931410296707
r2 manual: 0.8994931410296707
Fold 5 score: 0.8994931410296707
CPU times: user 20.4 s, sys: 3.15 s, total: 23.6 s
Wall time: 3.7 s


In [21]:
total_score = 0
for i, el in enumerate(scores_list):
    total_score += el['score']
    avg_score = total_score / (i+1)
print(avg_score)

0.911176214002537


In [22]:
0.911176214002537 - 0.911176214002537

0.0

# Grid Search CV for $\alpha$

In [23]:
%%time
lasso = Lasso()
# Initialize lists to store results
best_params_list = []
best_scores_list = []
coefficients_list = []
param_grid = {
    'alpha': np.linspace(0.001,400, 200)
}
# Perform Grid Search in each fold

# Configure GridSearchCV
grid_search = GridSearchCV(Lasso(), param_grid, cv=kf, scoring='r2', return_train_score=True)

grid_search.fit(x, y)
print('Best Estimator: ',grid_search.best_estimator_)
print('Best Params: ',grid_search.best_params_)
print('Best Score: ', grid_search.best_score_)


# Store best parameters and score
best_params_list.append({
    'params':grid_search.best_params_,
    # 'fold': fold_num,
    })
best_score = grid_search.best_score_
best_scores_list.append({
    'score':best_score,
    # 'fold':fold_num
    })
# Store coefficients of the best model
best_model = grid_search.best_estimator_
coefficients_list.append({
    'coef': best_model.coef_,
    # 'fold':fold_num
    })
print('Best Params List:', best_params_list)
print('Best Scores List:',best_scores_list)

Best Estimator:  Lasso(alpha=18.09140703517588)
Best Params:  {'alpha': 18.09140703517588}
Best Score:  0.915542051976342
Best Params List: [{'params': {'alpha': 18.09140703517588}}]
Best Scores List: [{'score': 0.915542051976342}]
CPU times: user 7min 18s, sys: 4min 25s, total: 11min 43s
Wall time: 1min 37s


In [24]:
cv_results_df = pd.DataFrame(grid_search.cv_results_)
cv_results_df.sort_values('mean_test_score', ascending=False).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
9,0.316764,0.066726,0.001932,0.001041,18.091407,{'alpha': 18.09140703517588},0.915465,0.914625,0.926296,0.914839,0.906486,0.915542,0.006309,1,0.936766,0.937177,0.935918,0.938062,0.939920,0.937569,0.001363
8,0.357375,0.074949,0.001409,0.000273,16.081362,{'alpha': 16.081361809045227},0.915981,0.914712,0.926181,0.914746,0.906060,0.915536,0.006397,2,0.937093,0.937472,0.936187,0.938312,0.940193,0.937851,0.001355
13,0.219259,0.051798,0.002175,0.001150,26.131588,{'alpha': 26.13158793969849},0.913709,0.914235,0.926558,0.915287,0.907765,0.915511,0.006115,3,0.935415,0.936082,0.934810,0.937018,0.938825,0.936430,0.001404
10,0.288941,0.064642,0.003860,0.003044,20.101452,{'alpha': 20.101452261306534},0.914835,0.914518,0.926415,0.914934,0.906819,0.915504,0.006265,4,0.936421,0.936885,0.935647,0.937793,0.939647,0.937279,0.001373
12,0.247641,0.045751,0.001957,0.000959,24.121543,{'alpha': 24.121542713567838},0.913978,0.914320,0.926529,0.915201,0.907480,0.915502,0.006160,5,0.935725,0.936330,0.935071,0.937269,0.939107,0.936700,0.001404


In [26]:
%%time
lasso = Lasso()
# Initialize lists to store results
best_params_list = []
best_scores_list = []
coefficients_list = []
param_grid = {
    'alpha': np.linspace(5,105, 200)
}
# Perform Grid Search in each fold

# Configure GridSearchCV
grid_search = GridSearchCV(Lasso(), param_grid, cv=kf, scoring='r2', return_train_score=True)

grid_search.fit(x, y)
print('Best Estimator: ',grid_search.best_estimator_)
print('Best Params: ',grid_search.best_params_)
print('Best Score: ', grid_search.best_score_)


# Store best parameters and score
best_params_list.append({
    'params':grid_search.best_params_,
    # 'fold': fold_num,
    })
best_score = grid_search.best_score_
best_scores_list.append({
    'score':best_score,
    # 'fold':fold_num
    })
# Store coefficients of the best model
best_model = grid_search.best_estimator_
coefficients_list.append({
    'coef': best_model.coef_,
    # 'fold':fold_num
    })
print('Best Params List:', best_params_list)
print('Best Scores List:',best_scores_list)

Best Estimator:  Lasso(alpha=17.060301507537687)
Best Params:  {'alpha': 17.060301507537687}
Best Score:  0.9155521869680454
Best Params List: [{'params': {'alpha': 17.060301507537687}}]
Best Scores List: [{'score': 0.9155521869680454}]
CPU times: user 14min 55s, sys: 5min 59s, total: 20min 55s
Wall time: 3min 5s


In [27]:
cv_results_df = pd.DataFrame(grid_search.cv_results_)
cv_results_df.sort_values('mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
24,0.352030,0.068466,0.002000,0.000914,17.060302,{'alpha': 17.060301507537687},0.915753,0.914671,0.926244,0.914808,0.906285,0.915552,0.006349,1,0.936929,0.937332,0.936058,0.938186,0.940061,0.937713,0.001359
23,0.384393,0.076305,0.001927,0.000591,16.557789,{'alpha': 16.55778894472362},0.915883,0.914693,0.926212,0.914790,0.906173,0.915550,0.006372,2,0.937009,0.937406,0.936124,0.938248,0.940126,0.937783,0.001356
25,0.355053,0.067206,0.003775,0.003654,17.562814,{'alpha': 17.562814070351756},0.915616,0.914648,0.926271,0.914822,0.906393,0.915550,0.006327,3,0.936850,0.937255,0.935990,0.938124,0.939993,0.937643,0.001361
26,0.364096,0.075244,0.003628,0.003144,18.065327,{'alpha': 18.065326633165828},0.915473,0.914626,0.926294,0.914838,0.906481,0.915542,0.006310,4,0.936770,0.937181,0.935921,0.938065,0.939924,0.937572,0.001363
22,0.367667,0.070125,0.001903,0.000936,16.055276,{'alpha': 16.055276381909547},0.915987,0.914712,0.926180,0.914744,0.906053,0.915535,0.006399,5,0.937098,0.937476,0.936190,0.938315,0.940197,0.937855,0.001355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.075825,0.014752,0.001568,0.000554,102.98995,{'alpha': 102.98994974874371},0.908982,0.911941,0.923740,0.914271,0.908193,0.913425,0.005593,196,0.928295,0.926167,0.925465,0.927856,0.927907,0.927138,0.001112
196,0.073369,0.013895,0.002222,0.001256,103.492462,{'alpha': 103.49246231155777},0.908970,0.911925,0.923735,0.914262,0.908164,0.913412,0.005599,197,0.928268,0.926121,0.925429,0.927807,0.927858,0.927097,0.001113
197,0.080452,0.012288,0.005066,0.005620,103.994975,{'alpha': 103.99497487437185},0.908959,0.911909,0.923731,0.914253,0.908136,0.913397,0.005605,198,0.928241,0.926075,0.925392,0.927757,0.927809,0.927055,0.001113
198,0.073064,0.012988,0.001431,0.000348,104.497487,{'alpha': 104.49748743718592},0.908947,0.911893,0.923726,0.914243,0.908109,0.913383,0.005611,199,0.928214,0.926028,0.925355,0.927709,0.927761,0.927014,0.001114


# Cross Validation

In [28]:
# Perform Cross Validation for the Model
model_cv_score = cross_val_score(Lasso(alpha= 17.060301507537687), x,y, cv=kf)
mean_model_score = np.mean(model_cv_score)
print('Each Fold Score:', model_cv_score)
print('Mean model score:', mean_model_score)
print('Score Variance:', np.var(model_cv_score))

Each Fold Score: [0.91575326 0.91467071 0.92624357 0.91480828 0.90628512]
Mean model score: 0.9155521869680454
Score Variance: 4.0311005671306246e-05


In [29]:
total_score = 0
for i, el in enumerate(scores_list):
    total_score += el['score']
    avg_score = total_score / (i+1)
print(avg_score)

0.911176214002537


In [30]:
mean_model_score - avg_score

0.004375972965508401

In [31]:
for el in coefficients_list:
    feature_name_and_coef = list(zip(list(scaled_df.columns), el['coef']))
    for feature, coef in feature_name_and_coef:
        print(f"{feature}: {coef}")
    print('-' * 40)  # Separator for readability between folds


GrLivArea: 24184.64219993617
LotFrontage: 987.0663976788387
LotArea: 4482.284949692567
OverallQual: 10734.189486363837
OverallCond: 6332.171834872125
YearBuilt: 9032.811095986639
YearRemodAdd: 611.6293807275235
MasVnrArea: 5872.815074034929
BsmtFinSF1: 10140.94186746271
BsmtFinSF2: 2698.9433452973403
BsmtUnfSF: -0.0
TotalBsmtSF: 7958.850047827229
1stFlrSF: 0.0
2ndFlrSF: 4193.857232131016
LowQualFinSF: -582.1665886579016
BsmtFullBath: 619.3124907016295
BsmtHalfBath: -262.2365807629696
FullBath: -73.6898273043039
HalfBath: 927.4287424366549
BedroomAbvGr: -4168.796872002005
KitchenAbvGr: -1515.21535067986
TotRmsAbvGrd: 2112.3747487795586
Fireplaces: 3042.3225976168715
GarageYrBlt: -3525.977543101741
GarageCars: 2622.9265728566666
GarageArea: 3449.0817424175875
WoodDeckSF: 813.5301233177239
OpenPorchSF: 82.51627053587457
EnclosedPorch: 354.7214186993397
3SsnPorch: 267.62840121322944
ScreenPorch: 2358.8255475806664
PoolArea: -2847.386606772908
MiscVal: 81.58726181544317
ExterQual: 3027.4920

# MAE calculation

In [37]:
lasso_mae = Lasso(alpha= 17.060301507537687)
lasso_mae.fit(X_train,y_train)
preds = lasso_mae.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 14487.598868108702


In [38]:
lasso_basic = Lasso()
lasso_basic.fit(X_train,y_train)
preds = lasso_basic.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 15014.759664906558


# Lasso For Feature Selection

In [32]:
lasso = Lasso(alpha = 100)
lasso.fit(x,y)
model_coef = lasso.coef_
len(model_coef)

293

In [33]:
len(scaled_df.columns)

293

In [34]:
list(zip(list(scaled_df.columns),model_coef))

[('GrLivArea', 25827.47988247478),
 ('LotFrontage', 1348.1496548705109),
 ('LotArea', 3732.364300245476),
 ('OverallQual', 12262.191388621322),
 ('OverallCond', 6363.43768447248),
 ('YearBuilt', 8676.300925233643),
 ('YearRemodAdd', 301.9144036742332),
 ('MasVnrArea', 6256.737528641779),
 ('BsmtFinSF1', 9598.36875209382),
 ('BsmtFinSF2', 2390.9978497068455),
 ('BsmtUnfSF', -0.0),
 ('TotalBsmtSF', 7448.058867131959),
 ('1stFlrSF', 0.0),
 ('2ndFlrSF', 467.2086218269908),
 ('LowQualFinSF', -757.8993525687633),
 ('BsmtFullBath', 715.496109164411),
 ('BsmtHalfBath', -369.23270372795747),
 ('FullBath', 0.0),
 ('HalfBath', 891.1351173557193),
 ('BedroomAbvGr', -4357.884194882711),
 ('KitchenAbvGr', -1277.4095834388263),
 ('TotRmsAbvGrd', 2135.0136652085876),
 ('Fireplaces', 2296.126467680957),
 ('GarageYrBlt', -2547.2717127107367),
 ('GarageCars', 1731.8488614242412),
 ('GarageArea', 4218.343405294109),
 ('WoodDeckSF', 929.1138727894983),
 ('OpenPorchSF', 129.05251340915763),
 ('EnclosedPorch